# Fase 2
NOTA:
- In questa fase, le immagini da inserire nel database saranno etichettate come image-X-Y- Z.png, dove
    - X ∈ {cc, con, detail, emboss, jitter, neg, noise1, noise2, original, poster, rot, smooth, stipple} denota il tipo dell’immagine;
    - 1≤Y≤40denota ilsubjectID,and
    - 1 ≤ Z ≤ 10 denota l’ ID del campione dell’immagine.
    

- I task in questa fase riguardano i tre modelli di feature e le misure di distanza/similarita’ sviluppate nella precedente.
 
- Potete usare le librerie esistenti per la decomposizione LDA.
- Potete usare le librerie esistenti per l’estrazione di autovalori e autovettori.

In [2]:
from ipynb.fs.full.task1_new import *
import numpy as np
from operator import itemgetter

from PIL import Image
from numpy import asarray
from scipy.stats import skew

# load and display an image with Matplotlib
from matplotlib import image
from matplotlib import pyplot
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from scipy.linalg import svd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
import re
from sklearn.utils.extmath import randomized_svd
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD

from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

import networkx as nx

import pickle


In [2]:
# import OS module
import os

# Get the list of all files and directories
path = "./secondaparte/images/"
dir_list = os.listdir(path) # array di stringhe contenente i nomi dei file
 
dir_list.sort()
dir_list.remove('.DS_Store')

print(len(dir_list))

4787


In [3]:
def createDictionaryAndWriteOnFile(matrix, filtered_dir_list, passedParameter, strategyName, modelName):
    matrixtrasposta = matrix.T

    dictionaryFeaturesLatenti = {}
    for i in range(len(matrixtrasposta)):
        sequenzaFeatureLatente = []
        for j in range(len(matrixtrasposta[i])):
            splittedString = re.split('-', filtered_dir_list[j])
            pair = ()

            if passedParameter == 'X':
                pair = (splittedString[2], matrixtrasposta[i][j])
            elif passedParameter == 'Y':
                pair = (splittedString[1], matrixtrasposta[i][j])
            
            sequenzaFeatureLatente.append(pair)
        
        sequenzaOrdinata = sorted(sequenzaFeatureLatente, key=lambda x: x[1], reverse=True)
        dictionaryFeaturesLatenti['Feature' + str(i)] = sequenzaOrdinata

    #print(dictionaryFeaturesLatenti)

    fileName = ''

    if passedParameter == 'X':
        fileName = 'VISUAL_task1_' + modelName + '_' + strategyName + '_coppie_soggetto-peso.txt'
    elif passedParameter == 'Y':
        fileName = 'VISUAL_task2_' + modelName + '_' + strategyName + '_coppie_tipo-peso.txt'

    # controlliamo se il file esiste o no
    file = open('./outputfilesecondaparte/' + fileName,'w+')

    with open('./outputfilesecondaparte/' + fileName, 'w') as convert_file:
        convert_file.write(json.dumps(dictionaryFeaturesLatenti, indent=4, sort_keys=True))

In [4]:
def createDictionaryAndWriteOnFileTask34(matrix, filtered_dir_list, passedParameter, strategyName, modelName):
    matrixtrasposta = matrix.T

    print(filtered_dir_list)
    print("LEN LIST", len(filtered_dir_list))
    print("len(matrixtrasposta)", len(matrixtrasposta))
    print("len(matrixtrasposta[0])", len(matrixtrasposta[0]))

    dictionaryFeaturesLatenti = {}
    for i in range(len(matrixtrasposta)):
        sequenzaFeatureLatente = []
        for j in range(len(matrixtrasposta[i])):
            pair = ()

            if passedParameter == 'soggetto':
                pair = (filtered_dir_list[j], matrixtrasposta[i][j])
            elif passedParameter == 'tipo':
                pair = (filtered_dir_list[j], matrixtrasposta[i][j])
            
            sequenzaFeatureLatente.append(pair)
        
        sequenzaOrdinata = sorted(sequenzaFeatureLatente, key=lambda x: x[1], reverse=True)
        dictionaryFeaturesLatenti['Feature' + str(i)] = sequenzaOrdinata

    print("DOPO IL FOR")

    #print(dictionaryFeaturesLatenti)

    fileName = ''

    if passedParameter == 'soggetto':
        fileName = 'VISUAL_task4_' + modelName + '_' + strategyName + '_coppie_soggetto-peso.txt'
    elif passedParameter == 'tipo':
        fileName = 'VISUAL_task3_' + modelName + '_' + strategyName + '_coppie_tipo-peso.txt'

    # controlliamo se il file esiste o no
    file = open('./outputfilesecondaparte/' + fileName,'w+')

    with open('./outputfilesecondaparte/' + fileName, 'w') as convert_file:
        convert_file.write(json.dumps(dictionaryFeaturesLatenti, indent=4, sort_keys=True))

In [5]:
def saveAllMatrixOnFile(u, s, vt, lista, strategyName, modelName, param, passedParameter):
    dictionary = {}

    if isinstance(u, list):
        dictionary["u"] = u
    else:
        dictionary["u"] = u.tolist()

    if isinstance(s, list):
        dictionary["s"] = s
    else:
        dictionary["s"] = s.tolist()

    if isinstance(vt, list):
        dictionary["vt"] = vt
    else:
        dictionary["vt"] = vt.tolist()

    dictionary["listaFiles"] = lista

    if passedParameter == 'X':
        fileName = 'LATENT_task1_' + modelName + \
            '_' + strategyName + '_' + param + '.txt'
    elif passedParameter == 'Y':
        fileName = 'LATENT_task2_' + modelName + \
            '_' + strategyName + '_' + param + '.txt'

    # controlliamo se il file esiste o no
    file = open('./outputfilesecondaparte/latent/' + fileName, 'w+')

    with open('./outputfilesecondaparte/latent/' + fileName, 'w') as convert_file:
        convert_file.write(json.dumps(dictionary, indent=4, sort_keys=True))


In [ ]:
def saveDecompositionOnFile(decomposition, strategyName, modelName, param, passedParameter):
    if passedParameter == 'X':
        fileName = 'LATENT_task1_' + modelName + \
            '_' + strategyName + '_' + param + '.pickle'
    elif passedParameter == 'Y':
        fileName = 'LATENT_task2_' + modelName + \
            '_' + strategyName + '_' + param + '.pickle'

    with open('./outputfilesecondaparte/latent/' + fileName, 'wb') as f:
        pickle.dump(decomposition, f)


def saveDecompositionOnFileTask34(decomposition, strategyName, modelName, param):
    if param == 'tipo':
        fileName = 'LATENT_task3_' + modelName + \
            '_' + strategyName + '_' + param + '.pickle'
    elif param == 'soggetto':
        fileName = 'LATENT_task4_' + modelName + \
            '_' + strategyName + '_' + param + '.pickle'

    with open('./outputfilesecondaparte/latent/' + fileName, 'wb') as f:
        pickle.dump(decomposition, f)

In [6]:
# funzione usata per ottenre le top-k semantiche latenti.
# parametri della funzione:
# modelName: nome del modello da usare (ColorMoments, LBP, HOG)
# X: il tipo dell'immagine
# k: numero delle semantiche da estrarre
# strategyName: nome della tecnica con cui su vuole effettuare la riduzione di dimensionalità (PCA, SVD, LDA)

def ottieniKSemanticheLatenti(modelName, param, k, strategyName, passedParameter):

    filtered_dir_list = []

    if passedParameter == 'X':
        filtered_dir_list = list(filter(lambda k: param in k, dir_list))
    elif passedParameter == 'Y':
        for i in range(len(dir_list)):
            splittedString = re.split('-', dir_list[i])

            if splittedString[2] == param:
                filtered_dir_list.append(dir_list[i])

    print("Numero di file su cui applichiamo i metodi: ", len(filtered_dir_list))
    #print(filtered_dir_list)
    dictionaryOfDescriptors = {}

    match modelName:
        case 'ColorMoments':
            print("SONO IN COLOR MOMENTS")
            # salvataggio dei descrittori dei color moments

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_ColorMoments.json')
            
            # returns JSON object as
            # a dictionary
            data = json.load(f)
            
            # Iterating through the json
            # list
            for key in data:
                #print(key)
                if key in filtered_dir_list:
                    dictionaryOfDescriptors[key] = data[key]
            
            # Closing file
            f.close()

        case 'LBP':
            print("SONO IN LBP")
            # salvataggio dei descrittori dei color moments

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_LBP.json')
            
            # returns JSON object as
            # a dictionary
            data = json.load(f)
            
            # Iterating through the json
            # list
            for key in data:
                #print(key)
                if key in filtered_dir_list:
                    dictionaryOfDescriptors[key] = data[key]
            
            # Closing file
            f.close()

        case 'HOG':
            
            print("HOG")

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_HOG.json')
            
            # returns JSON object as
            # a dictionary
            data = json.load(f)
            
            # Iterating through the json
            # list
            for key in data:
                #print(key)
                if key in filtered_dir_list:
                    dictionaryOfDescriptors[key] = data[key]
            
            # Closing file
            f.close()

        case _:
            return 4
            
    matrixObjFeatures = []

    lista = []

    for key in dictionaryOfDescriptors:
        matrixObjFeatures.append(dictionaryOfDescriptors[key])
        lista.append(key)

    print("Lunghezza di un descrittore: ", len(matrixObjFeatures[0]))
    print("Numero di oggetti: ", len(matrixObjFeatures))
    
    # calcoliamo il numero massimo di features latenti possibili, ovvero il minimo tra il numero di oggetti e di features di partenza
    numberOfFeatures = min(len(matrixObjFeatures[0]), len(matrixObjFeatures))

    match strategyName:
        case 'PCA':
            print("SONO IN PCA")

            numpyMatrixObjFeatures = np.array(matrixObjFeatures)

            covMatr = np.cov(numpyMatrixObjFeatures.T)

            print("\nNumero righe matrice di covarianza: ", len(covMatr))
            print("Numero colonne matrice di covarianza: ", len(covMatr[0]))
            print("La matrice di covarianza è quadrata? ", len(covMatr) == len(covMatr[0]))

            test_pca = PCA(n_components = numberOfFeatures)
            converted_data_test = test_pca.fit_transform(numpyMatrixObjFeatures)  # questa implementazione non richiede in input la matrice di covarianza

            createDictionaryAndWriteOnFile(converted_data_test, lista, passedParameter, strategyName, modelName)

            pca = PCA(n_components = numberOfFeatures)
            converted_data = pca.fit(numpyMatrixObjFeatures)  # questa implementazione non richiede in input la matrice di covarianza

            saveDecompositionOnFile(converted_data, strategyName, modelName, param, passedParameter)

            return 1

        case 'SVD':

            print("SONO IN SVD")

            numpyMatrixObjFeatures = np.array(matrixObjFeatures)

            test_svd = TruncatedSVD(algorithm = 'randomized', n_components = numberOfFeatures - 1)  # Abbiamo messo il -1 pechè voleva necessariamente un numero minore rispetto al numero di features iniziali
            converted_data_test = test_svd.fit_transform(numpyMatrixObjFeatures)  # questa implementazione non richiede in input la matrice di covarianza

            createDictionaryAndWriteOnFile(converted_data_test, lista, passedParameter, strategyName, modelName)

            svd = TruncatedSVD(algorithm = 'randomized', n_components = numberOfFeatures - 1)   # Abbiamo messo il -1 pechè voleva necessariamente un numero minore rispetto al numero di features iniziali
            converted_data = svd.fit(numpyMatrixObjFeatures)  # questa implementazione non richiede in input la matrice di covarianza

            saveDecompositionOnFile(converted_data, strategyName, modelName, param, passedParameter)

            return 2

        case 'LDA':
            print("SONO IN LDA")

            numpyMatrixObjFeatures = np.array(matrixObjFeatures)
            
            test_lda = LatentDirichletAllocation(n_components=numberOfFeatures,
                random_state=0)

            converted_data_test = test_lda.fit_transform(numpyMatrixObjFeatures)  # questa implementazione non richiede in input la matrice di covarianza

            createDictionaryAndWriteOnFile(converted_data_test, lista, passedParameter, strategyName, modelName)

            lda = LatentDirichletAllocation(n_components=numberOfFeatures,
                random_state=0)
            converted_data = lda.fit(numpyMatrixObjFeatures)  # questa implementazione non richiede in input la matrice di covarianza

            saveDecompositionOnFile(converted_data, strategyName, modelName, param, passedParameter)

            return 3

        case _:
            return 4


# Funzione per task 3 e 4

In [7]:
# funzione usata per ottenre le top-k semantiche latenti.
# parametri della funzione:
# modelName: nome del modello da usare (ColorMoments, LBP, HOG)
# X: il tipo dell'immagine
# k: numero delle semantiche da estrarre
# strategyName: nome della tecnica con cui su vuole effettuare la riduzione di dimensionalità (PCA, SVD, LDA)

def ottieniKSemanticheLatentiTask34(modelName, param, k, strategyName):

    dictionaryOfDescriptors = {}

    match modelName:
        case 'ColorMoments':
            print("SONO IN COLOR MOMENTS")
            # salvataggio dei descrittori dei color moments

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_ColorMoments.json')
            
            # returns JSON object as
            # a dictionary
            dictionaryOfDescriptors = json.load(f)
            
            # Closing file
            f.close()

        case 'LBP':
            print("SONO IN LBP")
            # salvataggio dei descrittori dei color moments

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_LBP.json')
            
            # returns JSON object as
            # a dictionary
            dictionaryOfDescriptors = json.load(f)
            
            # Closing file
            f.close()

        case 'HOG':
            
            print("HOG")

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_HOG.json')
            
            # returns JSON object as
            # a dictionary
            dictionaryOfDescriptors = json.load(f)
            
            # Closing file
            f.close()

        case _:
            return 4
            
    lista = []
    #dictionaryOfDescriptors contiene TUTTI i descrittori di TUTTI gli oggetti
    if param == 'tipo':
        lista = ['cc', 'con', 'emboss', 'jitter', 'neg', 'noise01', 'noise02', 'original', 'poster', 'rot', 'smooth', 'stipple']
        print(lista)

    elif param == 'soggetto':
        lista = []
        for i in range(40):
            lista.append(str(i)) #aggiungo il soggetto

        print(lista)
    else: 
        raise Exception("Param ha un valore sbagliato")
        

    # Per poter calcolare le matrici di similarità tipo-tipo o soggetto-soggetto abbiamo deciso di seguire il seguente approccio:
        # - presi tutti i descrittori delle immagini dello stesso tipo (o soggetto a seconda del caso) abbiamo calcolato il centroide,
        #   il quale viene usato come rappresentante del tipo (o soggetto)
        # - una volta calcolati tutti i centroidi abbiamo calcolato la similarità tra le varie coppie costruendo la matrice di similarità


    # Al posto di usare i centroidi avremmo potuto usare i medoidi: ovvero gli oggetti reali del Db che distano meno dagli altri.


    arrayDiCentroidi = []

    filteredFile = []
    for i in range(len(lista)):
        filteredDictionary = {}
        for key in dictionaryOfDescriptors:
            if lista[i] in key:
                filteredFile.append(key)
                filteredDictionary[key] = dictionaryOfDescriptors[key]
        
        matrix = []
        #TODO: calcolare il centroide
        for key in filteredDictionary:
            matrix.append(filteredDictionary[key])
        
        npMatrix = np.array(matrix)

        centroide = np.mean(npMatrix, axis=0)
        arrayDiCentroidi.append(centroide)

    print(arrayDiCentroidi)
    print(len(arrayDiCentroidi[0]))

    #ciclare tutti i centroidi e calcolare le distanze tra di essi

    matrixForDecomposition = np.eye(len(arrayDiCentroidi)) #matrice tipo-tipo o soggetto-soggetto

    for i in range(len(arrayDiCentroidi)):
        for j in range(i, len(arrayDiCentroidi)):
            similarity = cosine_similarity(arrayDiCentroidi[i].reshape(1,-1), arrayDiCentroidi[j].reshape(1,-1)) # calcolo la similarità tra i due descrittori
            matrixForDecomposition[i][j] = similarity
            matrixForDecomposition[j][i] = similarity

    print(matrixForDecomposition)
    print("QUA 0")

    numpyMatrixForDecomposition = np.array(matrixForDecomposition)
    print("QUA 1")

    # calcoliamo il numero massimo di features latenti possibili, ovvero il minimo tra il numero di oggetti e di features di partenza
    numberOfFeatures = min(len(numpyMatrixForDecomposition[0]), len(numpyMatrixForDecomposition))
    
    # print("FILTERED FILE LEN: ", len(filteredFile))
    # print(filteredFile)

    print("PRIMA DELLO SWITCH")

    match strategyName:
        case 'PCA':
            print("SONO IN PCA")

            numpyMatrixObjFeatures = np.array(numpyMatrixForDecomposition)

            covMatr = np.cov(numpyMatrixObjFeatures.T)

            print("\nNumero righe matrice di covarianza: ", len(covMatr))
            print("Numero colonne matrice di covarianza: ", len(covMatr[0]))
            print("La matrice di covarianza è quadrata? ", len(covMatr) == len(covMatr[0]))

            test_pca = PCA(n_components = numberOfFeatures)
            converted_data_test = test_pca.fit_transform(numpyMatrixObjFeatures)  # questa implementazione non richiede in input la matrice di covarianza

            createDictionaryAndWriteOnFileTask34(converted_data_test, lista, param, strategyName, modelName)

            pca = PCA(n_components = numberOfFeatures)
            converted_data = pca.fit(numpyMatrixObjFeatures)  # questa implementazione non richiede in input la matrice di covarianza

            saveDecompositionOnFileTask34(converted_data, strategyName, modelName, param)

            return 1

        case 'SVD':

            print("SONO IN SVD")

            numpyMatrixObjFeatures = np.array(numpyMatrixForDecomposition)

            test_svd = TruncatedSVD(algorithm = 'randomized', n_components = numberOfFeatures - 1)  # Abbiamo messo il -1 pechè voleva necessariamente un numero minore rispetto al numero di features iniziali
            converted_data_test = test_svd.fit_transform(numpyMatrixObjFeatures)  # questa implementazione non richiede in input la matrice di covarianza

            createDictionaryAndWriteOnFileTask34(converted_data_test, lista, param, strategyName, modelName)

            svd = TruncatedSVD(algorithm = 'randomized', n_components = numberOfFeatures - 1)   # Abbiamo messo il -1 pechè voleva necessariamente un numero minore rispetto al numero di features iniziali
            converted_data = svd.fit(numpyMatrixObjFeatures)  # questa implementazione non richiede in input la matrice di covarianza

            saveDecompositionOnFileTask34(converted_data, strategyName, modelName, param)

            return 2

        case 'LDA':
            print("SONO IN LDA")

            numpyMatrixObjFeatures = np.array(numpyMatrixForDecomposition)
            
            test_lda = LatentDirichletAllocation(n_components=numberOfFeatures,
                random_state=0)

            converted_data_test = test_lda.fit_transform(numpyMatrixObjFeatures)  # questa implementazione non richiede in input la matrice di covarianza

            createDictionaryAndWriteOnFileTask34(converted_data_test, lista, param, strategyName, modelName)

            lda = LatentDirichletAllocation(n_components=numberOfFeatures,
                random_state=0)
            converted_data = lda.fit(numpyMatrixObjFeatures)  # questa implementazione non richiede in input la matrice di covarianza

            saveDecompositionOnFileTask34(converted_data, strategyName, modelName, param)

            return 3

        case _:
            raise Exception("Metodo di decomposizione non valido")


> **Soluzione scelta:** Per ottenere le matrici di similarità tipo-tipo o soggetto-soggetto abbiamo recuperato tutte le immagini di quel tipo o di quel soggetto a seconda del caso e abbiamo calcolato per ogni tipo o soggetto un centroide.
Grazie a questi centroidi abbiamo poi costruito la matrice di similarità sfruttando la cosine similarity.

> **Nota bene:** Questa soluzione implementata per costruire la matrice di similarità è solo una delle tante disponibili. Avremmo potuto usare i medoidi oppure costruire una matrice per ogni tipo o soggetto e poi confrontare nella maniera corretta (perchè il confronto di matrici è soggetto all'ordine degli oggetti in esse) le matrici e calcolarne la distanza/similarit.

# Funzione per task 5

In [8]:
def getNsimilarImages(imageQueryFile, latentSemanticFileName, useCosineSimilarity):

    # apriamo l'immagine
    image = Image.open(imageQueryFile)

    try:
        splittedName = latentSemanticFileName.split("_")

        typeOfDescriptor = splittedName[2] # contiene il tipo di descrittore

        newImageDescriptor = ottieniDescrittoreOfAnyImages(imageQueryFile, typeOfDescriptor)

        with open(latentSemanticFileName, 'rb') as f:
            decomposition = pickle.load(f)

        tmp = decomposition.transform([newImageDescriptor])
        descriptorInNewSpace = tmp[0]
                    
        # Opening JSON file
        f = open('./databasefilesecondaparte/DB_' + typeOfDescriptor + '.json')
        
        # returns JSON object as
        # a dictionary
        data = json.load(f)

        mostSimilar = []

        # print("QUERY: ORA STAMPO IL DESCRITTORE RISPETTO ALLE K FEATURES LATENTI")
        # print(descriptorInNewSpace)

        for key in data:
            current_image = decomposition.transform([data[key]])
            currentDescriptorInNewSpace = current_image[0] 

            # print("OBJECT: ORA STAMPO IL DESCRITTORE RISPETTO ALLE K FEATURES LATENTI")
            # print(currentDescriptorInNewSpace)
                

            if useCosineSimilarity:
                distance = cosine_similarity(np.array(descriptorInNewSpace).reshape(1,-1), np.array(currentDescriptorInNewSpace).reshape(1,-1)) # calcolo la similarità tra i due descrittori
            else:        
                distance = np.linalg.norm(np.array(descriptorInNewSpace) - np.array(currentDescriptorInNewSpace))
            
            pair = (key, distance)

            mostSimilar.append(pair)

        #print("\nSTAMPO  IN ORDINE TUTTE LE IMMAGINI CON LA SIMILARITA'\n")
        
        if useCosineSimilarity:
            sequenzaOrdinata = sorted(mostSimilar, key=lambda x: x[1], reverse=True)
        else:
            sequenzaOrdinata = sorted(mostSimilar, key=lambda x: x[1])
        #print(sequenzaOrdinata)

    except:
        print("Il file di query e/o quello delle semantiche latenti non esiste! Inserire un nome corretto!")

    return sequenzaOrdinata

    # if done:
    #     # convertiamo l'immagine in un numpy array
    #     data = asarray(image)
    #     print(data)


# Task 6 e 7

In [ ]:


# import a groupby() method
# from itertools module
from itertools import groupby
  
# # dictionary
# INFO = [
#     {'employee': 'XYZ_1', 'company': 'ABC_1'},
#     {'employee': 'XYZ_2', 'company': 'ABC_2'},
#     {'employee': 'XYZ_3', 'company': 'ABC_3'},
#     {'employee': 'XYZ_4', 'company': 'ABC_3'},
#     {'employee': 'XYZ_5', 'company': 'ABC_2'},
#     {'employee': 'XYZ_6', 'company': 'ABC_3'},
#     {'employee': 'XYZ_7', 'company': 'ABC_1'},
#     {'employee': 'XYZ_8', 'company': 'ABC_2'},
#     {'employee': 'XYZ_9', 'company': 'ABC_1'}
# ]
  
# define a fuction for key
def key_func(k):
    return k[0]
  
# # sort INFO data by 'company' key.
# INFO = sorted(INFO, key=key_func)

def setClassForImage(imageQueryFile, latentSemanticFileName, nTask):

    print("1")
    done = False

    # apriamo l'immagine
    image = Image.open(imageQueryFile)
    print("2")
    # try:
    
    splittedName = latentSemanticFileName.split("_")
    print("3")
    typeOfDescriptor = splittedName[2] # contiene il tipo di descrittore
    print("4")
    newImageDescriptor = ottieniDescrittoreOfAnyImages(imageQueryFile, typeOfDescriptor)
    print("5")
    #print(newImageDescriptor)

    # file = open(latentSemanticFileName)
    # print("6")

    # # reading the data from the file
    # with open(latentSemanticFileName) as f:
    #     data = f.read()

    # dictionaryOfLatentSemantic = json.loads(data)

    # descriptorInNewSpace = []

    # for i in range(len(dictionaryOfLatentSemantic["vt"])):
    #     print("prima di coord: ", i)
    #     coord = np.asarray(dictionaryOfLatentSemantic["vt"][i]) @ np.asarray(newImageDescriptor)
    #     print("dopo coord: ", i)
    #     print("COORD: ", coord)

    #     descriptorInNewSpace.append(coord)
    #     print("dopo append: ", i)

    with open(latentSemanticFileName, 'rb') as f:
        decomposition = pickle.load(f)

    tmp = decomposition.transform([newImageDescriptor])
    descriptorInNewSpace = tmp[0]

    print("ORA STAMPO IL DESCRITTORE RISPETTO ALLE K FEATURES LATENTI")

    print(descriptorInNewSpace)

    # Opening JSON file
    f = open('./databasefilesecondaparte/DB_' + typeOfDescriptor + '.json')
    
    # returns JSON object as
    # a dictionary
    data = json.load(f)

    dictionaryOfDescriptors = {}

    for chiave, valore in data.items():
        splitted = str(chiave).split("-")

        if nTask == 6:
            key = splitted[1]
        else:
            key = splitted[2]

        dictionaryOfDescriptors[key] = []
    
    # print("SONO ARRIVATO QUA")
    # print(dictionaryOfDescriptors)

    for chiave, valore in data.items():
        splitted = str(chiave).split("-")

        if nTask == 6:
            key = splitted[1]
        else:
            key = splitted[2]

        newCurrentDescriptor = decomposition.transform([valore])

        tmp = dictionaryOfDescriptors[key]
        tmp.append(newCurrentDescriptor)
        dictionaryOfDescriptors[key] = tmp

    print("CICLO SUL DIZIONARIO")

    print(dictionaryOfDescriptors)

    print("LUNGHEZZA DIZIONARIO: ", len(dictionaryOfDescriptors))

    dictionaryOfCentroids = {}

    for chiave, value in dictionaryOfDescriptors.items():
        npMatrix = np.array(value)
        centroide = np.mean(npMatrix, axis=0)
        dictionaryOfCentroids[chiave] = centroide

    print("STAMPO IL DIZIONARIO DEI CENTROIDI")
    print(dictionaryOfCentroids)

    bestKey = None
    bestValue = None

    bestDistance = 99999999999999

    for chiave, value in dictionaryOfCentroids.items():
        # print("STO CICLANDO QUA")
        # print("VALUE HA LEN: ", len(value))
        # print("descriptorInNewSpace HA LEN: ", len(descriptorInNewSpace))

        distance = np.linalg.norm(np.array(value) - np.array(descriptorInNewSpace))
        
        if distance < bestDistance:
            bestDistance = distance
            bestKey = key
            bestValue = value

    print("LA BEST KEY E': ", bestKey)

    return bestKey

    # except:
        #print("Il file di query e/o quello delle semantiche latenti non esiste! Inserire un nome corretto!")

    

    # if done:
    #     # convertiamo l'immagine in un numpy array
    #     data = asarray(image)
    #     print(data)


# Funzione per task 8

In [9]:
# funzione usata per ottenre le top-k semantiche latenti.
# parametri della funzione:
# modelName: nome del modello da usare (ColorMoments, LBP, HOG)
# X: il tipo dell'immagine
# k: numero delle semantiche da estrarre
# strategyName: nome della tecnica con cui su vuole effettuare la riduzione di dimensionalità (PCA, SVD, LDA)

def createSimilarityMatrix(modelName, param):

    dictionaryOfDescriptors = {}

    match modelName:
        case 'ColorMoments':
            print("SONO IN COLOR MOMENTS")
            # salvataggio dei descrittori dei color moments

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_ColorMoments.json')
            
            # returns JSON object as
            # a dictionary
            dictionaryOfDescriptors = json.load(f)
            
            # Closing file
            f.close()

        case 'LBP':
            print("SONO IN LBP")
            # salvataggio dei descrittori dei color moments

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_LBP.json')
            
            # returns JSON object as
            # a dictionary
            dictionaryOfDescriptors = json.load(f)
            
            # Closing file
            f.close()

        case 'HOG':
            
            print("HOG")

            # Opening JSON file
            f = open('./databasefilesecondaparte/DB_HOG.json')
            
            # returns JSON object as
            # a dictionary
            dictionaryOfDescriptors = json.load(f)
            
            # Closing file
            f.close()

        case _:
            return 4
            
    lista = []
    #dictionaryOfDescriptors contiene TUTTI i descrittori di TUTTI gli oggetti
    if param == 'tipo':
        lista = ['cc', 'con', 'emboss', 'jitter', 'neg', 'noise01', 'noise02', 'original', 'poster', 'rot', 'smooth', 'stipple']
        print(lista)

    elif param == 'soggetto':
        lista = []
        for i in range(40):
            lista.append(str(i)) #aggiungo il soggetto

        print(lista)
    else: 
        raise Exception("Param ha un valore sbagliato")
        
    arrayDiCentroidi = []

    filteredFile = []
    for i in range(len(lista)):
        filteredDictionary = {}
        for key in dictionaryOfDescriptors:
            if lista[i] in key:
                filteredFile.append(key)
                filteredDictionary[key] = dictionaryOfDescriptors[key]
        
        matrix = []
        #TODO: calcolare il centroide
        for key in filteredDictionary:
            matrix.append(filteredDictionary[key])
        
        npMatrix = np.array(matrix)

        centroide = np.mean(npMatrix, axis=0)
        arrayDiCentroidi.append(centroide)

    print(arrayDiCentroidi)
    print(len(arrayDiCentroidi[0]))

    #ciclare tutti i centroidi e calcolare le distanze tra di essi

    matrixForDecomposition = np.eye(len(arrayDiCentroidi)) #matrice tipo-tipo o soggetto-soggetto

    for i in range(len(arrayDiCentroidi)):
        for j in range(i, len(arrayDiCentroidi)):
            similarity = cosine_similarity(arrayDiCentroidi[i].reshape(1,-1), arrayDiCentroidi[j].reshape(1,-1)) # calcolo la similarità tra i due descrittori
            matrixForDecomposition[i][j] = similarity
            matrixForDecomposition[j][i] = similarity

    print(matrixForDecomposition)
    
    return matrixForDecomposition

# Page Rank Algorithm

Link: https://www.geeksforgeeks.org/page-rank-algorithm-implementation/

In [10]:
def pagerank(G, alpha=0.85, personalization=None,
			max_iter=100, tol=1.0e-6, nstart=None, weight='weight',
			dangling=None):
	"""Return the PageRank of the nodes in the graph.

	PageRank computes a ranking of the nodes in the graph G based on
	the structure of the incoming links. It was originally designed as
	an algorithm to rank web pages.

	Parameters
	----------
	G : graph
	A NetworkX graph. Undirected graphs will be converted to a directed
	graph with two directed edges for each undirected edge.

	alpha : float, optional
	Damping parameter for PageRank, default=0.85.

	personalization: dict, optional
	The "personalization vector" consisting of a dictionary with a
	key for every graph node and nonzero personalization value for each node.
	By default, a uniform distribution is used.

	max_iter : integer, optional
	Maximum number of iterations in power method eigenvalue solver.

	tol : float, optional
	Error tolerance used to check convergence in power method solver.

	nstart : dictionary, optional
	Starting value of PageRank iteration for each node.

	weight : key, optional
	Edge data key to use as weight. If None weights are set to 1.

	dangling: dict, optional
	The outedges to be assigned to any "dangling" nodes, i.e., nodes without
	any outedges. The dict key is the node the outedge points to and the dict
	value is the weight of that outedge. By default, dangling nodes are given
	outedges according to the personalization vector (uniform if not
	specified). This must be selected to result in an irreducible transition
	matrix (see notes under google_matrix). It may be common to have the
	dangling dict to be the same as the personalization dict.

	Returns
	-------
	pagerank : dictionary
	Dictionary of nodes with PageRank as value

	Notes
	-----
	The eigenvector calculation is done by the power iteration method
	and has no guarantee of convergence. The iteration will stop
	after max_iter iterations or an error tolerance of
	number_of_nodes(G)*tol has been reached.

	The PageRank algorithm was designed for directed graphs but this
	algorithm does not check if the input graph is directed and will
	execute on undirected graphs by converting each edge in the
	directed graph to two edges.

	
	"""
	if len(G) == 0:
		return {}

	if not G.is_directed():
		D = G.to_directed()
	else:
		D = G

	# Create a copy in (right) stochastic form
	W = nx.stochastic_graph(D, weight=weight)
	N = W.number_of_nodes()

	# Choose fixed starting vector if not given
	if nstart is None:
		x = dict.fromkeys(W, 1.0 / N)
	else:
		# Normalized nstart vector
		s = float(sum(nstart.values()))
		x = dict((k, v / s) for k, v in nstart.items())

	if personalization is None:

		# Assign uniform personalization vector if not given
		p = dict.fromkeys(W, 1.0 / N)
	else:
		missing = set(G) - set(personalization)
		if missing:
			raise nx.NetworkXError('Personalization dictionary '
								'must have a value for every node. '
								'Missing nodes %s' % missing)
		s = float(sum(personalization.values()))
		p = dict((k, v / s) for k, v in personalization.items())

	if dangling is None:

		# Use personalization vector if dangling vector not specified
		dangling_weights = p
	else:
		missing = set(G) - set(dangling)
		if missing:
			raise nx.NetworkXError('Dangling node dictionary '
								'must have a value for every node. '
								'Missing nodes %s' % missing)
		s = float(sum(dangling.values()))
		dangling_weights = dict((k, v/s) for k, v in dangling.items())
	dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]

	# power iteration: make up to max_iter iterations
	for _ in range(max_iter):
		xlast = x
		x = dict.fromkeys(xlast.keys(), 0)
		danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
		for n in x:

			# this matrix multiply looks odd because it is
			# doing a left multiply x^T=xlast^T*W
			for nbr in W[n]:
				x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
			x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]

		# check convergence, l1 norm
		err = sum([abs(x[n] - xlast[n]) for n in x])
		if err < N*tol:
			return x
	raise nx.NetworkXError('pagerank: power iteration failed to converge '
						'in %d iterations.' % max_iter)

